In [73]:
# Set Envionmentals

# Imports
import os
from glob import glob
import pathlib
import json
import requests
from math import floor, ceil
#Third Party
import earthpy as et
import earthpy.appeears as etapp
# import earthpy.earthexplorer as 
import geopandas as gpd
import geoviews as gv
import hvplot.xarray
import numpy as np
import rioxarray as rxr
import rioxarray.merge as rxrmerge
from shapely.geometry import mapping
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import xarray as xr

with open('settings.json', 'r') as file:
    settings = json.load(file)

config = settings[0]

In [74]:
import math
import geopandas as gpd
from pyproj import CRS

def get_utm_crs(gdf):
    """Determine the UTM CRS based on the centroid of a GeoDataFrame."""
    # Ensure the gdf is in a geographic CRS for accurate centroid calculation
    gdf_geographic = gdf.to_crs(epsg=4326)
    centroid = gdf_geographic.geometry.unary_union.centroid
    lon, lat = centroid.x, centroid.y

    # Determine UTM zone
    utm_zone = math.floor((lon + 180) / 6) + 1

    utm_crs = CRS(f"EPSG:269{utm_zone:02d}")  # Northern hemisphere
    
     
    
    return utm_crs



In [75]:
# Get boundary gdf for clipping

# Download and load boundary data
boundary_config = config['boundary']
boundary_gdf = (gpd.read_file(boundary_config['url'])
                .to_crs(boundary_config['crs']))

# boundary_crs = get_utm_crs(boundary_gdf)
# print(boundary_crs)
# boundary_gdf = boundary_gdf.to_crs(boundary_crs)



study_site_gdf = (boundary_gdf
                  .set_index('GRASSLANDN')
                  .loc[[config['study_site']]]
                  )
print(study_site_gdf.crs)





EPSG:4269


In [76]:
(gv.tile_sources.EsriNatGeo *
 gv.Polygons(
     boundary_gdf
     [boundary_gdf.GRASSLANDN=='Kiowa National Grassland']
 )
 )


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (NATIONALGR,GRASSLANDN,GIS_ACRES,SHAPE_AREA,SHAPE_LEN)

In [77]:
print(study_site_gdf.crs)

EPSG:4269


In [78]:
from math import floor, ceil


def download_soil_data(study_site_gdf, config):
    raster_dir = "/workspaces/golden-feather-grass-habitats/assets/bin/rasters/soil"
    os.makedirs(raster_dir, exist_ok=True)


    bounds = study_site_gdf.total_bounds
    min_lon, min_lat, max_lon, max_lat = bounds

    min_lat, min_lon = floor(min_lat), floor(min_lon)
    max_lat, max_lon = ceil(max_lat), ceil(max_lon)

    soil = config['data_sources']['soil']
    property = soil['property']
    stat = soil['stat']
    depth = soil['depth']

    rasters = []
    for lat in range(min_lat, max_lat):
        for lon in range(min_lon, max_lon):
            polaris_url = (
                "http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/"
                "{property}/{stat}/{depth}/"
                "lat{min_lat}{max_lat}_lon{min_lon}{max_lon}.tif").format(
                    property=property, 
                    stat=stat, 
                    depth=depth,
                    min_lat=lat,
                    min_lon=lon,
                    max_lat=min(lat + 1, max_lat),
                    max_lon=min(lon + 1, max_lon))

            local_path = os.path.join(raster_dir, f"soil_data_{lat}_{lon}.tif")
            if not os.path.exists(local_path):
                response = requests.get(polaris_url)
                if response.status_code == 200:
                    with open(local_path, 'wb') as file:
                        file.write(response.content)
            rasters.append(local_path)

    # Merge rasters into a single data array
    merged_raster = None
    for raster in rasters:
        raster_data = rxr.open_rasterio(raster, masked=True).squeeze()
        if merged_raster is None:
            merged_raster = raster_data
        else:
            merged_raster = rxrmerge.merge_arrays([merged_raster, raster_data])

    # Reproject the raster to match the CRS of the study site GDF if they are different
    if merged_raster.rio.crs != study_site_gdf.crs:
        merged_raster = merged_raster.rio.reproject(study_site_gdf.crs)

    return merged_raster

# Usage
soil_data = download_soil_data(study_site_gdf, config)
soil_da = soil_data.rio.clip_box(*study_site_gdf.total_bounds)
soil_da


<xarray.DataArray (y: 2734, x: 5057)>
array([[8.183298 , 8.020073 , 8.026097 , ..., 7.6194496, 7.630881 ,
        7.8104725],
       [7.056039 , 8.227724 , 8.2896   , ..., 7.590799 , 7.725302 ,
        7.7987247],
       [7.02675  , 7.103374 , 7.103374 , ..., 7.75561  , 7.770836 ,
        7.7313895],
       ...,
       [7.792366 , 7.792366 , 6.69343  , ..., 8.14737  , 8.14737  ,
        8.14737  ],
       [7.792366 , 6.69343  , 6.69343  , ..., 8.14737  , 8.14737  ,
        8.114835 ],
       [6.69343  , 6.69343  , 7.792366 , ..., 8.14737  , 8.143957 ,
        8.13862  ]], dtype=float32)
Coordinates:
  * x            (x) float64 -104.4 -104.4 -104.4 ... -103.0 -103.0 -103.0
  * y            (y) float64 36.7 36.7 36.7 36.7 ... 35.94 35.94 35.94 35.94
    band         int32 1
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    _FillValue:     nan

In [79]:

# def get_srtm_data(study_site_gdf, config):
study_site_name = config['study_site']
download_key = study_site_name.replace("National Grassland", "SRTM").replace(" ", "-")
srtm_dir = config['data_sources']['elevation']['local_path']

    # Initialize the downloader
srtm_downloader = etapp.AppeearsDownloader(
    download_key=download_key,
    ea_dir=srtm_dir,
    product='SRTMGL1_NC.003',
    layer='SRTMGL1_DEM',
    start_date='02-11-2000',
    end_date='02-21-2000',
    polygon=study_site_gdf
    )

    # Download files if they don't already exist
if not os.path.exists(srtm_downloader.data_dir):
    srtm_downloader.download_files()

    # Find all downloaded SRTM files
srtm_paths = glob(os.path.join(srtm_dir, srtm_downloader.data_dir, '**', '*.tif'), recursive=True)

    # Load and merge the SRTM data arrays
print(srtm_paths)
srtm_da = [rxr.open_rasterio(srtm_path, masked=True).squeeze() for srtm_path in srtm_paths][0]
srtm_da = srtm_da.rio.reproject(study_site_gdf.crs)

srtm_da



['/workspaces/golden-feather-grass-habitats/assets/bin/rasters/DEM\\Kiowa-SRTM\\SRTMGL1_NC.003_2000001_to_2023352\\SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0001.tif', '/workspaces/golden-feather-grass-habitats/assets/bin/rasters/DEM\\Kiowa-SRTM\\SRTMGL1_NUMNC.003_2000001_to_2023352\\SRTMGL1_NUMNC.003_SRTMGL1_NUM_doy2000042_aid0001.tif']


<xarray.DataArray (y: 2734, x: 5057)>
array([[2059., 2057., 2056., ..., 1475., 1474., 1473.],
       [2061., 2059., 2056., ..., 1473., 1473., 1473.],
       [2063., 2060., 2058., ..., 1473., 1473., 1473.],
       ...,
       [1735., 1731., 1729., ..., 1292., 1290., 1287.],
       [1736., 1734., 1733., ..., 1292., 1292., 1289.],
       [1738., 1737., 1736., ..., 1291., 1293., 1291.]], dtype=float32)
Coordinates:
  * x            (x) float64 -104.4 -104.4 -104.4 ... -103.0 -103.0 -103.0
  * y            (y) float64 36.7 36.7 36.7 36.7 ... 35.94 35.94 35.94 35.94
    band         int32 1
    spatial_ref  int32 0
Attributes:
    add_offset:     0.0
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    units:          Meters

In [80]:
from xrspatial import slope
import rioxarray

# Ensure srtm_da is loaded correctly
if srtm_da is None:
    print("Error: srtm_da is not loaded correctly.")
else:
    # Calculate slope
    dem_slope = slope(srtm_da)

    # Check if dem_slope is computed correctly
    if dem_slope is None:
        print("Error: Failed to compute slope.")
    else:
        # Save the slope data as a new raster file
        slope_path = config['data_sources']['elevation']['slope_path']
        dem_slope.rio.to_raster(slope_path, driver='GTiff')
        print(f"Slope raster saved to {slope_path}")


Slope raster saved to /workspaces/golden-feather-grass-habitats/assets/bin/rasters/DEM/slope.tiff


In [83]:
# Get precipiaiton model for CONUS in year 1950
maca_url = ('http://thredds.northwestknowledge.net:8080/thredds/ncss/'
                'agg_macav2metdata_pr_bcc-csm1-1-m_r1i1p1_historical_1950_2005_CONUS_monthly.nc'
                '?var=precipitation'
                '&disableLLSubset=on&disableProjSubset=on'
                '&horizStride=1'
                '&time_start=1950-01-15T00%3A00%3A00Z&time_end=1950-12-15T00%3A00%3A00Z'
                '&timeStride=1&accept=netcdf')
maca_response = requests.get(maca_url)

# Grab the directory path from json
climate_path = config['data_sources']['climate']['local_path']

# Create the directory if it doesn't exist
if not os.path.exists(climate_path):
     os.makedirs(climate_path, exist_ok=True)

 # Define the full path including the filename
maca_path = os.path.join(climate_path, 'maca.nc')

 # Assuming maca_response is obtained from a requests.get() call
maca_response = requests.get(maca_url)

 # Write the file to the specified directory
with open(maca_path, 'wb') as maca_file:
    maca_file.write(maca_response.content)


In [84]:
maca_ds = xr.open_dataset(maca_path, engine = 'netcdf4')
maca_ds = maca_ds.assign_coords(lon=maca_ds.lon - 360)
maca_ds = maca_ds.rio.write_crs(4269)
precip_da = maca_ds['precipitation'].mean("time")
# precip_da = maca_ds.precipitation.mean("time")
precip_da.rio.write_crs(4269, inplace = True)
precip_da.rio.set_spatial_dims('lon','lat', inplace = True)

maca_ds.precipitation.mean('time').hvplot(rasterize = True)


BokehModel(combine_events=True, render_bundle={'docs_json': {'3cb7db7c-b42c-471c-a530-7f0186bd80ee': {'version…

In [85]:
precip_da = precip_da.rio.reproject_match(soil_da)
precip_da = precip_da.rio.clip_box(*study_site_gdf.total_bounds)
precip_da
print(precip_da.shape)

(2734, 5057)


c:\Users\Degg\miniconda3\envs\earth-analytics-python\lib\site-packages\rioxarray\raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


In [86]:
import rioxarray as rxr
from rasterio.enums import Resampling

def harmonize_rasters(raster_list, reference_raster):
    """
    Harmonize a list of rasters to match the extent, resolution, and CRS of a reference raster.
    Only reprojects and resamples rasters if they are not already harmonized.

    Parameters:
    raster_list (list of xarray.DataArray): List of rasters to be harmonized.
    reference_raster (xarray.DataArray): The raster to use as the reference for harmonization.

    Returns:
    list of xarray.DataArray: List of harmonized rasters.
    """
    harmonized_rasters = []
    for raster in raster_list:
        # Check if the raster is already harmonized with the reference raster
        if (raster.rio.crs == reference_raster.rio.crs and
            raster.rio.shape == reference_raster.rio.shape and
            raster.rio.transform() == reference_raster.rio.transform()):
            # Raster is already harmonized
            harmonized_rasters.append(raster)
        else:
            # Reproject to match CRS of reference raster
            reprojected_raster = raster.rio.reproject_match(reference_raster)

            # Resample to match resolution of reference raster, using nearest neighbor interpolation
            resampled_raster = reprojected_raster.rio.reproject(
                reference_raster.rio.crs,
                shape=(reference_raster.rio.height, reference_raster.rio.width),
                resampling=Resampling.nearest)

            harmonized_rasters.append(resampled_raster)

    return harmonized_rasters


# Now harmonized_rasters[0], harmonized_rasters[1], and harmonized_rasters[2]
# correspond to harmonized elevation, slope data, and climate respectively.


In [87]:
# Assuming rasters are  xarray.DataArray objects

rasters = [soil_da, srtm_da, dem_slope, precip_da]
raster_names = ['soil_da', 'srtm_da', 'dem_slope', 'precip_da']
# Check if any raster data array is None
for raster, name in zip(rasters, raster_names):
    if raster is None:
        print(f"Error: {name} is not loaded correctly.")

# If all rasters are loaded correctly, proceed with harmonization
if all(raster is not None for raster in rasters):
    harmonized_rasters = harmonize_rasters(
        [srtm_da, dem_slope, precip_da],
        reference_raster=soil_da
    )
    # Process harmonized rasters
else:
    print("Harmonization skipped due to missing raster data.")

c:\Users\Degg\miniconda3\envs\earth-analytics-python\lib\site-packages\rioxarray\raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


In [89]:
print(harmonized_)

AttributeError: 'list' object has no attribute 'name'

In [ ]:
# import json
# import numpy as np
# import skfuzzy as fuzz
# from skfuzzy import control as ctrl

# # Load the configuration
# with open('settings.json', 'r') as file:
#     config = json.load(file)

# # Access the first element of the list (which is a dictionary)
# config_dict = config[0]

# # Now access the fuzzy_logic section
# fuzzy_logic_config = config_dict['fuzzy_logic']

# # Define fuzzy variables based on JSON configuration
# variables = {}
# for var_name, var_details in fuzzy_logic_config['variables'].items():
#     range_min, range_max, range_step = var_details['range']
#     antecedent = ctrl.Antecedent(np.arange(range_min, range_max, range_step), var_name)
#     for set_name, set_details in var_details['sets'].items():
#         antecedent[set_name] = fuzz.trimf(antecedent.universe, set_details['params'])
#     variables[var_name] = antecedent

# # Continue with the rest of your code...

# # Continue with the rest of your code...

# # Define fuzzy variables based on JSON configuration
# variables = {}
# for var_name, var_details in config_dict['fuzzy_logic']['variables'].items():
#     range_min, range_max, range_step = var_details['range']
#     antecedent = ctrl.Antecedent(np.arange(range_min, range_max, range_step), var_name)
#     for set_name, set_details in var_details['sets'].items():
#         antecedent[set_name] = fuzz.trimf(antecedent.universe, set_details['params'])
#     variables[var_name] = antecedent

# # Define output variable for habitat suitability
# habitat_suitability = ctrl.Consequent(np.arange(0, 100, 1), 'habitat_suitability')
# habitat_suitability['low'] = fuzz.trimf(habitat_suitability.universe, [0, 25, 50])
# habitat_suitability['medium'] = fuzz.trimf(habitat_suitability.universe, [25, 50, 75])
# habitat_suitability['high'] = fuzz.trimf(habitat_suitability.universe, [50, 75, 100])

# def create_rule(condition, outcome):
#     # Split the condition into parts and handle logical operators
#     condition_parts = condition.split()
#     parsed_condition = []
#     for part in condition_parts:
#         if part in ["AND", "OR"]:
#             parsed_condition.append(part.lower())  # Convert to Python logical operators
#         else:
#             var_name, set_name = part.split('.')
#             parsed_condition.append(f"variables['{var_name}']['{set_name}']")
#     parsed_condition_str = " ".join(parsed_condition)

#     # Construct the rule using the parsed condition
#     rule_condition = eval(parsed_condition_str)
#     return ctrl.Rule(rule_condition, habitat_suitability[outcome])

# # Now create the rules using the updated create_rule function
# rules = []
# for rule_config in config_dict['fuzzy_logic']['rules']:
#     condition = " ".join(rule_config['if'])
#     outcome = rule_config['then'].split('.')[-1]  # Extract the outcome part
#     rule = create_rule(condition, outcome)
#     rules.append(rule)

# # Continue with the rest of your code...


# # Control system
# habitat_ctrl = ctrl.ControlSystem(rules)
# habitat_simulation = ctrl.ControlSystemSimulation(habitat_ctrl)




In [ ]:
# # Define the order of variables as they appear in harmonized_rasters
# variable_order = ['soil_pH', 'elevation', 'climate', 'slope']

# try:
#     # Apply fuzzy logic to each raster dataset
#     for variable_name, raster_data in zip(variable_order, harmonized_rasters):
#         if variable_name in variables:
#             # Assuming you want to use the mean value of the raster
#             mean_value = raster_data.mean()
#             habitat_simulation.input[variable_name] = mean_value
#         else:
#             print(f"Variable '{variable_name}' not found in fuzzy logic configuration.")

#     # Compute the result
#     habitat_simulation.compute()

#     # Extract the output
#     habitat_suitability_result = habitat_simulation.output['habitat_suitability']

#     # Visualization and further processing
#     # ...
# except Exception as e: 
#     print(f"Error in applying fuzzy logic: {e}")


In [90]:
import xarray as xr
import numpy as np

# Create a simple test data array
x = np.linspace(0, 10, 100)
test_data_array = xr.DataArray(x, dims=["x"])

# Define a simple triangular membership function
def create_membership_function(data_array, a, b, c):
    return xr.where(
        (data_array <= a) | (data_array >= c), 0,
        xr.where(
            data_array <= b,
            (data_array - a) / (b - a), 
            (c - data_array) / (c - b))
    )

# Apply the membership function to the test data array
mf_test = create_membership_function(test_data_array, 3, 5, 7)

# Define and apply a simple fuzzy rule
def apply_simple_rule(data_array, threshold):
    return xr.where(data_array > threshold, 1, 0)

# Apply the rule to the membership function result
rule_result = apply_simple_rule(mf_test, 0.5)

# Print the results
print("Membership Function Result:\n", mf_test)
print("\nRule Application Result:\n", rule_result)


Membership Function Result:
 <xarray.DataArray (x: 100)>
array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01515152, 0.06565657, 0.11616162, 0.16666667, 0.21717172,
       0.26767677, 0.31818182, 0.36868687, 0.41919192, 0.46969697,
       0.52020202, 0.57070707, 0.62121212, 0.67171717, 0.72222222,
       0.77272727, 0.82323232, 0.87373737, 0.92424242, 0.97474747,
       0.97474747, 0.92424242, 0.87373737, 0.82323232, 0.77272727,
       0.72222222, 0.67171717, 0.62121212, 0.57070707, 0.52020202,
       0.46969697, 0.41919192, 0.36868687, 0.31818182, 0.26767677,
       0.21717172, 0.16666667, 0.11616162, 0.06565657, 0.01515152,
     

In [91]:
def create_membership_function(data_array, set_config):
    set_type = set_config['type']
    a, b, c = set_config['params']  # Unpack the parameters directly
    if set_type == 'triangular':
        return xr.where(
            (data_array <= a) | (data_array >= c), 0,
            xr.where(
                data_array <= b,
                (data_array - a) / (b - a), 
                (c - data_array) / (c - b))
        )


In [92]:
def apply_membership_functions(variables_data, variables_config):
    membership_functions = {}
    for var_name, data_array in variables_data.items():
        membership_functions[var_name] = {}
        for set_name, set_config in variables_config[var_name]['sets'].items():
            membership_functions[var_name][set_name] = create_membership_function(data_array, set_config)
    return membership_functions


In [ ]:
# Load your fuzzy logic configuration
fuzzy_config = config['fuzzy_logic']

# Prepare your data arrays
variables_data = {
    'soil_pH': soil_da,
    'elevation': srtm_da,
    'climate': precip_da,
    'slope': dem_slope
}

# Apply fuzzy logic to the entire arrays
habitat_suitability = apply_fuzzy_rule(variables_data, 
                                       fuzzy_config['variables'], 
                                       fuzzy_config['rules'])


# Visualization and further processing...


In [104]:
def evaluate_fuzzy_rules(membership_functions, rules_config):
    # Initialize habitat suitability with zeros
    reference_mf = next(iter(next(iter(membership_functions.values())).values()))
    habitat_suitability = xr.full_like(reference_mf, 0)

    # Iterate over each rule
    for rule in rules_config:
        condition = rule['if']
        outcome = rule['then'].split('.')[-1]  # Extract the outcome part

        

        # Initialize a temporary array for this rule's result
        rule_result = xr.full_like(reference_mf, 0)

        # Process the condition
        for i in range(0, len(condition), 2):
            var_set = condition[i].split('.')
            mf = membership_functions[var_set[0]][var_set[1]]
            print("rule_result dimensions:", rule_result.dims)
            print("rule_result coordinates:", rule_result.coords)

            print("mf dimensions:", mf.dims)
            print("mf coordinates:", mf.coords)
            aligned_rule_result, aligned_mf = xr.align(rule_result, mf, join='outer')

            print("Aligned rule_result dimensions:", aligned_rule_result.dims)
            print("Aligned rule_result coordinates:", aligned_rule_result.coords)

            print("Aligned mf dimensions:", aligned_mf.dims)
            print("Aligned mf coordinates:", aligned_mf.coords)


            if i == 0:
                rule_result = mf
            else:
                if condition[i-1] == 'AND':
                    rule_result = xr.where(aligned_rule_result > 0, aligned_mf, 0)
                elif condition[i-1] == 'OR':
                    rule_result = np.maximum(aligned_rule_result, aligned_mf)

        # Apply the outcome to the habitat suitability
        if outcome == 'high':
            habitat_suitability = np.maximum(habitat_suitability, rule_result * 100)
        elif outcome == 'medium':
            habitat_suitability = np.maximum(habitat_suitability, rule_result * 50)
        elif outcome == 'low':
            habitat_suitability = np.maximum(habitat_suitability, rule_result * 25)

    return habitat_suitability

# Evaluate the fuzzy rules with the full logic
habitat_suitability = evaluate_fuzzy_rules(membership_functions, fuzzy_config['rules'])

# Print the final habitat suitability result
print("Habitat Suitability Result:", habitat_suitability)


rule_result dimensions: ('y', 'x')
rule_result coordinates: Coordinates:
  * x            (x) float64 -104.4 -104.4 -104.4 ... -103.0 -103.0 -103.0
  * y            (y) float64 36.7 36.7 36.7 36.7 ... 35.94 35.94 35.94 35.94
    band         int32 1
    spatial_ref  int32 0
mf dimensions: ('y', 'x')
mf coordinates: Coordinates:
  * x            (x) float64 -104.4 -104.4 -104.4 ... -103.0 -103.0 -103.0
  * y            (y) float64 36.7 36.7 36.7 36.7 ... 35.94 35.94 35.94 35.94
    band         int32 1
    spatial_ref  int32 0
Aligned rule_result dimensions: ('y', 'x')
Aligned rule_result coordinates: Coordinates:
  * x            (x) float64 -104.4 -104.4 -104.4 ... -103.0 -103.0 -103.0
  * y            (y) float64 36.7 36.7 36.7 36.7 ... 35.94 35.94 35.94 35.94
    band         int32 1
    spatial_ref  int32 0
Aligned mf dimensions: ('y', 'x')
Aligned mf coordinates: Coordinates:
  * x            (x) float64 -104.4 -104.4 -104.4 ... -103.0 -103.0 -103.0
  * y            (y) float64 36

In [106]:
# Convert DataArray to hvplot image
habitat_suitability.hvplot.image(
    x='x', y='y', 
    cmap='viridis', 
    width=700, height=400, 
    colorbar=True, 
    title='Habitat Suitability Map'
)*study_site_gdf.hvplot.image()


AttributeError: 'GeoDataFrame' object has no attribute 'hvplot'

In [98]:
# Simplified rule evaluation for testing
def evaluate_fuzzy_rules_test(membership_functions):
    # Just use one variable and one set for testing
    test_mf = membership_functions['soil_pH']['acidic']
    habitat_suitability = xr.full_like(test_mf, 0)

    # Apply a simple rule for testing
    habitat_suitability = xr.where(test_mf > 0.5, 100, habitat_suitability)

    return habitat_suitability

# Test the simplified rule evaluation
habitat_suitability_test = evaluate_fuzzy_rules_test(membership_functions)
print("Habitat Suitability Test Result:", habitat_suitability_test)


Habitat Suitability Test Result: <xarray.DataArray (y: 2734, x: 5057)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
  * x            (x) float64 -104.4 -104.4 -104.4 ... -103.0 -103.0 -103.0
  * y            (y) float64 36.7 36.7 36.7 36.7 ... 35.94 35.94 35.94 35.94
    band         int32 1
    spatial_ref  int32 0


In [97]:
# Harmonize rasters (excluding soil_da which is the reference)
harmonized_rasters = harmonize_rasters([srtm_da, dem_slope, precip_da], reference_raster=soil_da)
print("Harmonized Rasters Shapes:", [raster.shape for raster in harmonized_rasters])

# Prepare data arrays including soil_da
variables_data = {
    'soil_pH': soil_da,  # Reference raster, already harmonized
    'elevation': harmonized_rasters[0],
    'climate': harmonized_rasters[1],
    'slope': harmonized_rasters[2]
}

# Print shapes of the data arrays
print("Data Arrays Shapes:")
for var_name, data_array in variables_data.items():
    print(f"{var_name}: {data_array.shape}")

# Apply membership functions
fuzzy_config = config['fuzzy_logic']
membership_functions = apply_membership_functions(variables_data, fuzzy_config['variables'])

# Print the membership functions dictionary
print("Membership Functions Dictionary:")
for var_name, sets in membership_functions.items():
    print(f"Variable: {var_name}")
    for set_name, mf in sets.items():
        print(f"  {set_name}: {mf}")

# Check the output of next(iter(membership_functions.values()))
print("\nOutput of next(iter(membership_functions.values())):")
first_mf_output = next(iter(membership_functions.values()))
print(first_mf_output)


# Print membership function results
print("Membership Functions Results:")
for var_name, mf_dict in membership_functions.items():
    print(f"{var_name}:")
    for set_name, mf in mf_dict.items():
        print(f"  {set_name}: {mf}")

# Evaluate fuzzy rules
habitat_suitability = evaluate_fuzzy_rules(membership_functions, fuzzy_config['rules'])

# Print habitat suitability result
print("Habitat Suitability Result:", habitat_suitability)


c:\Users\Degg\miniconda3\envs\earth-analytics-python\lib\site-packages\rioxarray\raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


Harmonized Rasters Shapes: [(2734, 5057), (2734, 5057), (2734, 5057)]
Data Arrays Shapes:
soil_pH: (2734, 5057)
elevation: (2734, 5057)
climate: (2734, 5057)
slope: (2734, 5057)
Membership Functions Dictionary:
Variable: soil_pH
  acidic: <xarray.DataArray (y: 2734, x: 5057)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
  * x            (x) float64 -104.4 -104.4 -104.4 ... -103.0 -103.0 -103.0
  * y            (y) float64 36.7 36.7 36.7 36.7 ... 35.94 35.94 35.94 35.94
    band         int32 1
    spatial_ref  int32 0
  slightly_acidic: <xarray.DataArray (y: 2734, x: 5057)>
array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       

TypeError: Expected DataArray, Dataset, or Variable